# Example for Downloading Dataset using Magnet Link

### Requirements

In [ ]:
!python -m pip install --upgrade pip setuptools wheel
!python -m pip install lbry-libtorrent
!apt install python3-libtorrent

### Initialize Libtorrent Downloader Object

In [ ]:
import libtorrent as lt

# Download session
ses = lt.session()
# Specify port for listening the connection (port_1) and alternate port (port_2)
port_1 = 6881
port_2 = 6891
ses.listen_on(port_1, port_2)
# Placeholder for torrent or magnet link
downloads = []

### Specify Destination Path for Download

In [ ]:
import os

# Mount Google Drive (optional), the destination path for download can also be session storage.
from google.colab import drive
drive.mount._DEBUG = False
drive.mount('/content/drive', force_remount=True)

# Destination path for download
destination_path = './downloads'
os.makedirs(destination_path, exist_ok=True)

### Specify Magnet Link and Perform Download

In [ ]:
# Specify the magnet link for dataset download
magnet_link ='magnet:?'

# Session parameter
params = {'save_path': destination_path}
downloads.append(lt.add_magnet_uri(ses, magnet_link, params))


import time
from IPython.display import display
import ipywidgets as widgets


# Status string
state_str = [
    "queued",
    "checking",
    "downloading metadata",
    "downloading",
    "finished",
    "seeding",
    "allocating",
    "checking fastresume",
]

# Progress bar widgets
layout = widgets.Layout(width="auto")
style = {"description_width": "initial"}
download_bars = [
    widgets.FloatSlider(
        step=0.01, disabled=True, layout=layout, style=style
    )
    for _ in downloads
]
display(*download_bars)

while downloads:
    next_shift = 0
    for index, download in enumerate(downloads[:]):
        bar = download_bars[index + next_shift]
        if not download.is_seed():
            s = download.status()

            bar.description = " ".join(
                [
                    download.name(),
                    str(s.download_rate / 1000),
                    "kB/s",
                    state_str[s.state],
                ]
            )
            bar.value = s.progress * 100
        else:
            next_shift -= 1
            ses.remove_torrent(download)
            downloads.remove(download)
            bar.close() # Seems to be not working in Colab (see https://github.com/googlecolab/colabtools/issues/726#issue-486731758)
            download_bars.remove(bar)
            print(download.name(), "complete")
    time.sleep(1)